In [32]:
import xgboost as xgb
import boto3
boto3.setup_default_session(profile_name='ml-staging-admin')
# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client(
    "sagemaker-runtime", region_name='us-east-1',)

# The endpoint name must be unique within 
# an AWS Region in your AWS account. 
endpoint_name='fraud-detect-xgb-endpoint'



In [16]:
payload = "2,0,0,1,58400,14247.76686667085,72647.76686667085,7,29,0,11,41,108,0,1,750,3000,0,2,2014,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1\n2,0,2,0,11500,10675.671346738158,22175.67134673816,11,28,3,16,54,171,0,1,750,2750,2,3,2015,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1\n2,0,0,1,18500,10202.266353999776,28702.26635399977,1,6,6,8,51,196,0,1,750,3000,0,1,2011,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0\n1,0,0,0,16300,9338.348066435718,25638.34806643572,9,1,6,3,57,122,0,1,750,3000,1,1,2015,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0\n2,1,0,0,14700,28145.92499418225,42845.92499418225,10,23,2,15,59,142,0,1,750,3000,2,3,2019,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0\n"


response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=payload,
        ContentType="text/csv",
        Accept="text/csv"
    )

In [18]:
response

{'ResponseMetadata': {'RequestId': '992e7fad-810a-4684-b3b4-2c9d534f59ad',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '992e7fad-810a-4684-b3b4-2c9d534f59ad',
   'x-amzn-invoked-production-variant': 'Alltraffic',
   'date': 'Sun, 18 Feb 2024 23:51:07 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '99',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'Alltraffic',
 'Body': <botocore.response.StreamingBody at 0x1172aea30>}

In [20]:
response_body_str = response['Body'].read().decode('utf-8')
response_body_str

'0.02243666537106037\n0.02243666537106037\n0.0799826979637146\n0.13931523263454437\n0.03112351894378662\n'

In [25]:
import pickle as pkl 
import tarfile

model = pkl.load(open("xgboost-model", 'rb'))


[00:07:07] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:1203: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[00:07:07] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[00:07:07] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:553: 
  If you are loading a serialized model 

In [30]:
import numpy as np
data_single = [2,0,0,1,58400,14247.76686667085,72647.76686667085,7,29,0,11,41,108,0,1,750,3000,0,2,2014,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1]
data_single_2d = np.array(data_single).reshape(1, -1)
dmat_single = xgb.DMatrix(data_single_2d)
model.predict(dmat_single)

array([0.02243667], dtype=float32)

In [33]:
# with my own model
data = np.arange(10).reshape(1, 10).astype(float)

# Convert the numpy array into a DMatrix
dmat = xgb.DMatrix(data)

In [36]:
data

array([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [31]:
# trying to load the model

model = xgb.Booster()
model.load_model("../model.bin")
model



In [34]:
model.predict(dmat)

array([0.981199], dtype=float32)

In [38]:
payload = "0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0\n"
endpoint_name='xgboostTest-endpoint'

response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=payload,
        ContentType="text/csv",
        Accept="text/csv"
    )

In [39]:
response_body_str = response['Body'].read().decode('utf-8')
response_body_str

'0.9811990261077881\n'